In [1]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
import folium
import warnings
import sys
warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

Geopandas has version 0.13.2
Movingpandas has version 0.17.1


In [2]:
# read data from file
filename = '../data/processed/202204_points_stavanger_cleaned_200k.parquet'
# filename = '../../data/processed/202204_points_stavanger_cleaned_full.parquet'
gdf = gpd.read_parquet(filename)
gdf.head()

,mmsi,imo_nr,length,lon,lat,sog,cog,true_heading,nav_status,message_nr,geometry,speed
date_time_utc,,,,,,,,,,,,
2022-04-01 06:30:21,209989000_0,9235505,90,4.62360,59.5881,10.0,167.2,174,0,1,POINT (4.62360 59.58810),4.473722
2022-04-01 06:30:31,209989000_0,9235505,90,4.62367,59.5877,9.7,179.6,174,0,1,POINT (4.62367 59.58770),4.473722
2022-04-01 06:30:40,209989000_0,9235505,90,4.62375,59.5873,9.9,173.0,174,0,1,POINT (4.62375 59.58730),4.976744
2022-04-01 06:30:50,209989000_0,9235505,90,4.62384,59.5868,9.8,174.7,174,0,1,POINT (4.62384 59.58680),5.593419
2022-04-01 06:31:10,209989000_0,9235505,90,4.62402,59.5859,9.7,177.4,174,0,1,POINT (4.62402 59.58590),5.038954


In [3]:
# convert to Trajectory Collection
trajectories = mpd.TrajectoryCollection(gdf, traj_id_col='mmsi', obj_id_col='mmsi')

print(f'Loaded dataset: {filename}')
print(f'AIS messages: {len(gdf)}')
print(f'Trajectories: {len(trajectories)}')

Loaded dataset: ../data/processed/202204_points_stavanger_cleaned_200k.parquet
AIS messages: 192346
Trajectories: 249


In [4]:
# compute median sampling interval
sampling_intervals = []
for trajectory in trajectories:
    sampling_intervals.append(trajectory.get_sampling_interval().total_seconds())
print(f'Median sampling interval of all trajectories: {np.median(np.array(sampling_intervals))} seconds')

Median sampling interval of all trajectories: 10.0 seconds


In [5]:
# Douglas Peucker trajectory generalization to reduce the number of trajectory points (for plotting purposes)
simplified_trajectories = mpd.DouglasPeuckerGeneralizer(trajectories).generalize(tolerance=0.0005)
n_points, n_DP_points = len(gdf), len(simplified_trajectories.to_point_gdf())
print(f'DP reduced {n_points} AIS messages to {n_DP_points} points ({n_DP_points/n_points*100:.2f}%)')

DP reduced 192346 AIS messages to 5863 points (3.05%)


In [9]:
# plot n random trajectories against the DP simplified trajectories
plot_comparison = True
if plot_comparison:
    n_trajectories = 5  # -1 selects all trajectories
    columns = ['mmsi', 'geometry']  # columns to be plotted
    selection = np.random.randint(0, high=len(trajectories), size=n_trajectories)
    map = trajectories.to_traj_gdf()[columns].iloc[selection].explore(cmap='jet', column='mmsi', name='Trajectories', 
                                                                      style_kwds={'opacity':0.5, 'weight':1})
    map = simplified_trajectories.to_traj_gdf()[columns].iloc[selection].explore(m=map, cmap='jet', column='mmsi', 
                                                                                 name='Simplified trajectories')
    folium.LayerControl().add_to(map)
map